In [ ]:
# Standard imports
import numpy as np
from itertools import chain, combinations

# Data manipulation
import pandas as pd
import polars as pl

# Sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score,cross_validate, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.compose import ColumnTransformer


# Scipy packages
from scipy.stats import zscore
from scipy.special import factorial
import scipy.optimize as so

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Supervised Learning

## Data preprocessing

Common computing paradigms: Rules + Data = Answers (Through Computing)

Machine Learning: Data + Answers = Rules (Learning!)

Using data is a process that repeats in a cycle:

1. Business understanding
2. Data understanding
    - Refer back to **1.** to ensure cohesiveness
    - Aiming to select variables and clean data (filter outliers, irrelevant variables, delete null values)
3. Data preparation
4. Modeling
    - Refer back to **3.** to ensure cohesiveness
5. Evaluation
    - Return back to **1.** if needed
6. Deployment

Steps to cleaning data:

1. Create variables
2. Eliminate redundant variables
3. Treat null values
4. Treat outliers as valid or invalid

**Why do we want to clean data?**

- Dirty data that doesn't make sense in our context (Eg, age being negative, is 0 a true value or absence of something, incomplete data, duplicate)
- Missing values **can be kept** since it may be valuable. It is good practice to create an additional category or indicator variable for them
- Missing values **should be deleted** when there are too many missing values
- Missing values **can be replaced** when you can estimate them using imputation procedures
    - For *continuous variables*, replace with median or mean (median more robust to outliers)
    - For *nominal variables*, replace with mode
    - For *regression* or *tree-based imputation*, predict using other variables; cannot use target class as predictor if missing values can occur during model usage
- Outliers **can be kept** as null values or **deleted** when they are invalid observations or out of distribution due to low sample value
    - Some may be hidden in one-dimensional views of multidimensional data
    - Invalid observations are typically treated as null values, while out of distribution are typically removed and a note is made in the policy
- **Valid outliers** and their treatments
    - Truncation based on z-scores: replace all variable values with z-scores above or below 3 with the mean $\pm$ 3 standard deviations
    - Truncation based on inter-quartile range: replace all variable values with $Median \pm 3s, \text{ where s }= IQR/(2 \times 0.6745)$
    - Truncation using a sigmoid

## Statistical models

### Input data

There are multiple types of numerical data

- Continuous, categorical, integer

In general, we will assume that for each individual sample $i$, there is a vector $x_i$ that stores the information of $p$ variables that represent that individual. We have a sample $X$, the design matrix, of cardinality $I$ such that for every $i$ we know their information $x_i$

### Target Variables

The core of *supervised learning* is that there is a variable $y$ representing the outcome

- Continuous $y$ is a supervised regression problem
- Discrete $y$ is a supervised classification problem
- Binary $y$ is the easiest version of a supervised classification problem

If no set $Y$ outcomes exist, then the problem is *unsupervised*

## Defining a model and losses

A statistical model takes an input and gives an output. A model minimizes loss or error function by measuring the goodness of fit of the function. There are many examples of loss minimizing functions, such as *mean-square error* or *cross-entropy*(MLE)

**Mean-square error**: $L(Y,\hat{Y}) = \sum_i (y_i - \hat{y}_i)^2$

- Appropriate for regression analysis

**Cross entropy**: When you have no understanding, then entropy (chaos) is at its worst. Each additional piece of information is worth less than the one before.

We start with one hot vector $y_i$ where it is 1 if in the correct class and 0 otherwise. There are a total of $k$ classes

Eg, If you have 3 classes with probabilities $P_A, P_B, P_C$ with a response vector $y_i = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}$ and $f(x_i) = (P_a, P_b, P_c)$,

- You can isolate the probabilities by setting $(P_a^{y_a} \times P_b^{y_b} \times P_c^{y_c})$
- Thus, the loss function is: $L(y_i, \hat{y}_i) = -\sum_k y_i^k log\hat{(y_i^k)}$
    - If $P_n$ is 0, then we accept that $0log(0) = 0$

$l(y_i, \hat{y}_i) = -y_ilog(\hat{y}_i) - (1-y_i)log(1-\hat{y}_i)$ for a binary version

$l(y_i, \hat{y}_i) = - \sum_k y_i^k log\hat{(y_i^k)}$ for multinomial cross-entropy, where $k$ represents the number of vectors

### Regression functions

The "best" model is the one that minimizes the expected error over the full distribution of the set of input and output

The problem, mathematically, becomes $min_\beta E[L(Y,f(x|\beta))]$

**Linear regression example**

For the whole sample:

$$Y = X\beta + \epsilon$$

We can then take the least-squares to find the model for our $\beta$



# Likelihood

"Given a distribution with parameters $\theta$, how likely are my data?"

We assume that:

- Assume points are independent
- Can be used with any distribution

## Properties of MLE

- Functional invariance
    - $y=f(\theta) \rightarrow \hat y=f(\hat \theta)$
- Asymptotic properties
    - Estimate is asymptotically unbiased
    - Estimate is asymptotically efficient (the best model)
    - Estimate is asymptotically normall distributed
- If from the normal distribution, the variance MLE is the biased sample variance (dividing by n)



# Logistic regression

With logistic regression, we can identify how every variable impacts the model.

### Classification and regression

When you use least-squares, there is no guarantee that your response is between 0 and 1. Also, your target must be normally distributed

Logistic regression eliminates ambiguity by pushing middle cases to 0 or 1.
    - Intuitively, the odds of an event grow exponentially

MLE for logistic regression is **supervised**, so it should have a target. Cases in the sample are independent and identically distributed. iid



# Model validation

We can compare supervised models using test data. The model with better generalizable characteristics will be better. 

- we can use the mse, rmse, mae, and the mean relative error as performance measures

## Training, validation, and test set

1. training set is to calibrate the parameters
2. validation set: part of training, but used to make decisions of model construction
    - Deciding what variables
    - Deciding when to stop training
3. test set: **Only** used to calculate final metrics. No decision should be made on the test set

## Misuse of the test set

- Using it to help build your model, using it to remove variables and calibrate the model
    - Leads to overly optimistic models
- To do with test set:
    1. Split at the very beginning
    2. Only decision should be model selection
    3. If you need to go back to a previous step, resplit training/validation/test set
        - Usually not feasible and expensive, but can have serious consequences if leakage occurs
- **MUST replace test set null values with train set median/mean/parameter**, or else there **WILL BE LEAKAGE**

## Conditional vs expected test error

The **conditional test error** calculates the expectation over different test sets, given a specific training data set. Specific to a given model over a population. 

- Typically looked at more because it is a fixed model

The **expected test error** is taken over both training and test sets, is an average over different model configurations

# Decomposing prediction error

1. Irreducible error: variability around the true relationship between $x$ and $y$
    - This is the best test error we can expect
2. Bias: systematic difference of the best fitted model from the true relationship
    - $E[\hat f(x_i)] - f(x_i)$
    - Occurs when the function is not the same class of functions fitted
    - **Universal approximation property**: some models can approximate any function with enough data: neural networks and tree ensembles
3. Variance: variance around the average fit
    - $E[\hat f(x_i) - E[\hat f(x_i)]]^2$
    - If the model is too complex, variance will skyrocket

# Bias-variance trade-off

We will see that although a more complex model is better for training data, it will not be generalizable for other data sets

## Overfitting

This is when you have made training error near 0 and thus test error increases a lot

Your model should be the best for the data you have and the problem you are trying to solve

## Size of the test set

- Big enough to detect a difference in test error of interest
- Small enough to leave enough data for model fitting
    - Rule of thumb: 30% of full data should be allocated to the test data

# Measurement in practice

1. Performance: how well does the estimated model perform with new observations?
2. Decide on how to split the data up: 
    - Split sample
    - N-fold cross validation
    - Single sample
    - Bootstrapping
3. Decide on the performance measure to use

## Splitting the data

### Split sample method

- For large data sets with more than 1000 obs, more than 50 groups is sufficient
- Distribution of classes should be the same in training and test sets
- We need STRICT separation between test and training data

### N-fold cross-validation

- Typically for smaller data sets under 1000 observations

1. Split data into 'N' folds
2. Train on 'N-1' folds and test on the remaining fold
3. Repeat, where the test set changes every time, thus repeating 'N' times

## Out of sample, out of time, out of universe quantitative validation

1. Out of sample: test and training data is mixed on one plane
    - When mixed with out of time, the data is taken at a different time, so separate
2. Out of universe: training and test sets are separated, parallel across time
3. Out of time: test set is taken after training or vice versa

## Classic performance measures

### confusion matrix

- can calculate things like sensitivity, specificity, PPV, NPV

### ROC curve

- AUC larger is better
- For multiple classes
    - One vs all approach: calculate overall AUC as the weighted average of individual 'n' AUCs
    - One vs one approach: calculate nC2 AUCs, then take the average


In [ ]:
diabetes = pd.read_csv("diabetes.csv")
diabetes.drop(diabetes[ (diabetes['BMI'] == 0) & (diabetes['BloodPressure'] == 0)].index, inplace=True) # Cannot have 0 BMI, ones with 0 blood pressure do not have useful information
diabetes['Glucose'] = diabetes['Glucose'].replace(0,diabetes['Glucose'].median()) # Cannot have 0 Glucose, rows still have other information
diabetes['BloodPressure'] = diabetes['BloodPressure'].replace(0,diabetes['BloodPressure'].median()) # Cannot have 0 blood pressure
diabetes['BMI'] = diabetes['BMI'].replace(0,diabetes['BMI'].median()) # Cannot have 0 BMI

In [ ]:
posrate = diabetes.Outcome.mean() # 1-baseline accuracy
baselineacc = 1-posrate
print(f"The baseline accuracy of the data is {baselineacc}") 

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(diabetes.drop('Outcome',axis='columns'), # splitting into train and test based on the 'outcome' column
                                                diabetes['Outcome'], # what is considered the response variable
                                                test_size=0.3, # percentage of observations dedicated o the test set
                                                random_state=42,  # random state that ensures your results are replicable
                                                stratify=diabetes['Outcome']) # stratify by the response variable

In [ ]:
columns_to_transform = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
                        'Insulin', 'BMI', 'Age', 'DiabetesPedigreeFunction'] # sets the names of the columns we need to use

transform_columns = ColumnTransformer([('scaler', StandardScaler(), columns_to_transform)], # scales the columns such that they can all be used in our pipeline
                                      remainder='passthrough', # for columns that don't need to be transformed, they are left in our set of columns rather than dropped
                                      verbose_feature_names_out=False) # are not given a prefix

logit_pipe = Pipeline([ # trains a pipeline to fit a logistic regression model
    ('scaler', transform_columns), 
    ('logistic_regression', LogisticRegression(solver='lbfgs', # default solver for the logistic regression function
                                               penalty = None, # no penalty like ridge or lasso
                                               max_iter=1000, # number of iterations
                                               verbose=1, # given verbosity since positive
                                               random_state=42, # random seed
                                               n_jobs=-1, # use all processors
                                               class_weight='balanced')) # adjusts weights so they are inversely proportional to the frequency
])

# Train the pipeline

logit_pipe.fit(Xtrain, ytrain) # give the pipeline data to train with

In [ ]:
print(classification_report(ytest, logit_pipe.predict(Xtest))) # assesses the accuracy of the trained model, using the test set
accuracy = logit_pipe.score(Xtest, ytest.to_numpy().ravel()) # also another way of taking the accuracy of the trained model

In [ ]:
y_test_prediction = logit_pipe.predict_proba(Xtest) # the model values for the logistic regression
fpr, tpr, thresholds = roc_curve(y_true=ytest, y_score=y_test_prediction[:,1]) # creating the roc curve using the true values vs the model's values
auc = np.round(roc_auc_score(y_true=ytest, y_score=y_test_prediction[:,1]), # calculating AUC, again using the true vs predicted values
               decimals = 3)
confusion_matrix_log = confusion_matrix(ytest, logit_pipe.predict(Xtest)) # creating a confusion matrix using a function
sensitivity = confusion_matrix_log[1,1] / (confusion_matrix_log[1,1] + confusion_matrix_log[1,0]) # calculating sensitivity using the confusion matrix
specificity = confusion_matrix_log[0,0] / (confusion_matrix_log[0,0] + confusion_matrix_log[0,1]) # calculating specificity using the confusion matrix


# Plot ROC curve
plt.plot(sensitivity, label=f"Sensitivity = {sensitivity:.3f}")
plt.plot(specificity, label=f"Specificity = {specificity:.3f}")
plt.plot(fpr,tpr,label=f"AUC = {auc:.3f}")
plt.legend(loc=2)
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC Curve of Logistic Regression Model')
plt.show()

# Uncertainty management

## Estimation and sampling distribution

Parameters characterize the population we want to study, like expectations or values that describe the relationship between input and output

- Eg, mean, median, slope

Sampling distributions are the distribution of an estimator

- How good is the estimate of the population parameter?

## Confidence intervals

For a random variable with a normal distribution, the sample and the population have different variance, thus when we use a confidence interval, we use the sampling standard error. 

A confidence interval states that for every $\alpha$%, we expect a certain number out of 100 iterations to contain our true mean, variance, median, etc

- Variances however require a chi-squared distribution
- Confidence intervals do not work with any ranked measures (AUC, ROC curve)

## The bootstrap

1. Take a sample of the population
2. Resample with replacement an equal sized sample from your original sample. 
3. Your bootstrap statistics will differ each time
    - For a statistic $\theta$, the distribution of the difference between the original sample statistic and your bootstrapped statistics is asymptotically equal to the real difference between the population statistic and your original sample statistic
        - $\delta_i^\star = \theta^\star - \theta_i^\star$
    - We can estimate the confidence intervals using the quantiles of $\delta$

The bootstrap is a universal technique to obtain confidence intervals and can be applied to any statistics. The confidence intervals are asymptotically correct and the bootstrap does not make assumptions about the underlying distribution

Make sure to save checkpoints and your model outputs so you don't have to reload your training each time.

- Can be very unstable when the population is small
- Takes time to compute
- Applicable when measure is not normally distributed and for complex statistics

## Parameter uncertainty

Calculating confidence intervals of performance measures:

1. Take a trained model
2. calculate the bootstrap interval over samples with repetitions of the test set
3. Take the confidence interval of that specific model's test distribution, use the center as the original test set's estimate

calculating confidence intervals of parameter estimates: using bootstrap to calculate regression parameters

1. Take a dataset and calculate the bootstrapped samples from one original sample
2. Train the full pipeline over the bootstrapped sample
    - splitting train and test sets
    - Normalize data
    - Train the model
    - calculate the performance
3. calculate the confidence interval for the parameters using the original training parameter estimate

## Bootstrap vs cross-validation

- Bootstrap is more precise for most cases
- If there are less cases than variables, then cross-validation is more robust
- At minimum, run 10 by 10 CV, but better is to run 100 by 100
    - CV is cheaper than bootstrap
- Cross-validation is to tune hyperparameters and select the best model

## Prediction uncertainty

How certain can we be with a point estimate? We can calculate a confidence interval based on each bootstrap estimate

In [ ]:
yprob = logit_pipe.predict_proba(Xtest)[:, 1] # Get the predicted probabilities of the test data

ypred = logit_pipe.predict(Xtest) # Get the predicted classes of the test data

n_bootstraps = 1000 # set the bootstraps amounts
bootstrapped_accuracy = np.zeros(n_bootstraps)
bootstrapped_auc = np.zeros(n_bootstraps)

for i in range(n_bootstraps):
    idx = np.random.choice(len(ytest), len(ytest), replace=True) # randomly selecting the a set with the length of the test set from the test set with replacement

    bootstrapped_accuracy[i] = accuracy_score(ytest.to_pandas().iloc[idx], ypred[idx])  # Get the accuracy of the bootstrap sample

    bootstrapped_auc[i] = roc_auc_score(ytest.to_pandas().iloc[idx], yprob[idx]) # Get the AUC of the bootstrap sample
    
# Get the differences between the bootstrapped values and the original values
accuracy_diff = bootstrapped_accuracy - accuracy
auc_diff = bootstrapped_auc - auc

# Calculate the 95% confidence interval for the accuracy and AUC
accuracy_ci = np.percentile(accuracy_diff, [2.5, 97.5])
auc_ci = np.percentile(auc_diff, [2.5, 97.5])

# Show the ci bounds for the pipeline developed through bootstrapping
print(accuracy_ci)
print(auc_ci)

In [ ]:
logit_pipe1 = Pipeline([ # creating another pipeline using logistic regression
    ('scaler', transform_columns),
    ('logistic_regression', LogisticRegression(solver='lbfgs',
                                               penalty = None,
                                               max_iter=10000,
                                               verbose=0,
                                               random_state=0,
                                               n_jobs=1,
                                               class_weight='balanced'))
])

cv = RepeatedKFold(n_splits=10, n_repeats=100, random_state=0) # creating a random set for cross-validation

cv_results = cross_validate(logit_pipe1, # The pipeline to cross-validate
                            diabetes.drop('Outcome'), # The features
                            diabetes.select('Outcome').to_numpy().ravel(), # The target
                            cv=cv, # The cross-validation object we created
                            scoring=['accuracy', 'roc_auc'], # The metrics we want to calculate
                            return_estimator=True, # Return the estimator for each fold. Useful for calculating parameter uncertainty.
                            n_jobs=-1 # Use all available cores
                            )

logit_pipe1.fit(Xtrain, ytrain.to_numpy().ravel()) # training the pipeline
accuracy_scores = cv_results['test_accuracy'] # obtaining the trained pipeline accuracy using the cross-validation function we defined above
auc_scores = cv_results['test_roc_auc'] # obtaining the trained pipeline AUC using the cross-validation function we defined above

# Row and columnar data formats

A new data type that was specially formulated for data scientists

## How data is stored

Data is stored linearly by the computer in a string of zeroes and ones. This is called tabular data in memory

We must orient our data to make it make sense to the computer: 

- Row data storage
- Columnar data storage

### Row storage (for operational databases)

Used in databases, essentially each row from a table all next to each other in one line. Since it is in one row, it is expensive to search all data and replace or delete

Eg, Name|Age|Place|Name|Age|Place|Name|Age|Place|

Advantages:

- each value in the same row are close in space
- deleting a row is easy, same as inserting a new row
- searching is easy to find all the data within a single row
- accessing data is easier in rows (random access)

Disadvantages:

- Expensive and inefficient for analytics

### Columnar storage (Superior data method)

Each column from a table is next to each other, sequentially. Is much more efficient for calculating over columns. It also has less data space on your disk

Eg, Name|Name|Name|Age|Age|Age|Place|Place|Place

Advantages:

- Single instruction operations are very efficient
- Modifying columns is faster
- Uncompressed data is more efficient
- Compressed data is more efficient
- Allows for sinlge instruction, multiple data operations of contiguous column data
- Data for each column is stored contiguously in memory

## Columnar vs row data formats

For operational databases, we should stick to **row formats**

- Random access is needed
- We can delete and edit with ease

For data science pipelines, we should use **columnar data**

- We normally do operations over rows
- Space is an issue
- We rarely delete or add data

## Spark / Arrow / DuckDB

**Apache Parquet** (in disk) and **Apache Arrow** (in memory) are two softwares that store data as columnar

Implementations

- DuckDB: traditional databases with columnar storage, optimized for data warehousing and storage
- Polars: Library for data manipulation, well-structured API that is expressive and easy to use

# Feature selection and regularization

## Approaches to feature selection

1. Subset selection: identifying a subset of predictors that we believe is related to the response
2. Shrinkage/regularization: fitting a model with all predictors but shrinking the coefficients
3. Dimension reduction

### Sequential selection: greedy searches

1. forward selection: start from 0 predictors, then add one each time until the next one we add doesn't help us
    - May miss the best model since it doesn't consider the interactions between the predictors
2. backward selection: start with all, take off predictors until we cannot feasibly do so anymore
    - Usually ends up with more predictors than forward selection
3. Stepwise selection: evaluates whether we should add or subtract predictors at each step

Greedy searches do not lead to a globally optimal solution

### Regularization/shrinkage

These approaches work on penalties when the model uses more variables than necessary

We define the loss as:

$$L(x,y| f) = L_{Bias}(x,y|f) + \lambda L_{variance}(x,y|f)$$

$lambda$ is a **hyperparameter** that can be set to define any configurable part of a model's learning process

- It changes the function itself, the outcome, and capacity of the model
- It is a modeler's choice and depends on the problem
- The $\lambda$ must be tuned using cross-validation
- As $\lambda$ increases, the variance decreases and the bias increases

#### Ridge regression (L2 norm)

Imposes a squared loss: $\lambda L_{variance}(x,y|f) = \lambda \sum_{k=1}^V \beta_k^2$

- variables must be standardized to ensure the model converges to a solution
- Does not select variables, but shrinks them to near 0 instead. 
- Eliminates correlations between variables

#### Lasso regression (L1 norm)

Imposes an absolute value penalty: $\lambda L_{variance}(x,y|f) = \lambda \sum_{k=1}^V |\beta_k|$

- can eliminate variables - variable selection
- Needs cleaning
- Coefficients go to 0 very fast
- does not allow for correlation between variables
- variables need to be normalized, and it is much more sensitive method to de-normalized variables
- good for industries with lots of variables and not that much data

#### Elasticnet

Imposes an average between ridge and lasso: $\lambda L_{variance}(x,y|f) = \lambda \bigg( \alpha\sum_{k=1}^V |\beta_k| + \frac{1-\alpha}{2} \sum_{k=1}^V \beta_k^2\bigg)$

- $\alpha$ is a balance parameter between the weight given to LASSO vs ridge
- Typically want to give more weight to LASSO than ridge, unless data is highly correlated
- Make sure to read the documentation of elastic net in python since it is not written as it is above
